<a href="https://colab.research.google.com/github/nvinogradskaya/DL_HW4_RNN/blob/main/LSTM%2BContrastive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Concatenate, Input, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json

In [3]:
from google.colab import drive
SEQ_LENGTH = 10
EMBEDDING_DIM = 16
LSTM_UNITS = 64
BATCH_SIZE = 64
EPOCHS = 5
TEST_SIZE = 0.2

drive.mount('/content/drive')
DATA_PATH = "/content/drive/My Drive/Colab Notebooks/Data/"
SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/contrastive_results/"
os.makedirs(SAVE_PATH, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def calculate_metrics(preds, targets):
    # Средняя ошибка смещения (ADE)
    ade = np.mean(np.linalg.norm(preds - targets, axis=-1))

    # Финальная ошибка смещения (FDE)
    fde = np.linalg.norm(preds[:, -1] - targets[:, -1], axis=-1).mean()

    # Accuracy@1
    distances = np.linalg.norm(preds[:, None] - targets[:, :, None], axis=-1)
    acc1 = np.mean(np.argmin(distances, axis=-1) == 0)

    return ade, fde, acc1

In [5]:
def load_and_preprocess_data(data_path, max_users=10):
    data = []
    user_dirs = sorted(os.listdir(data_path))[:max_users]

    for user in user_dirs:
        traj_dir = os.path.join(data_path, user, 'Trajectory')
        traj_files = [f for f in os.listdir(traj_dir) if f.endswith('.plt')]

        for traj_file in traj_files:
            df = pd.read_csv(
                os.path.join(traj_dir, traj_file),
                skiprows=6,
                header=None,
                usecols=[0, 1, 3, 5, 6],
                names=['lat', 'lon', 'alt', 'date', 'time']
            )
            df['user'] = user
            data.append(df)

    df = pd.concat(data, ignore_index=True)
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.sort_values(by=['user', 'datetime'], inplace=True)
    df = df[(df['lat'] != 0) & (df['lon'] != 0)].ffill()

    scaler = MinMaxScaler()
    df[['lat', 'lon', 'alt']] = scaler.fit_transform(df[['lat', 'lon', 'alt']])

    df['hour_sin'] = np.sin(2 * np.pi * df['datetime'].dt.hour / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['datetime'].dt.hour / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['datetime'].dt.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['datetime'].dt.dayofweek / 7)

    user_ids = {user: idx for idx, user in enumerate(df['user'].unique())}
    df['user_id'] = df['user'].map(user_ids)

    return df, user_ids, scaler

In [6]:
df, user_ids, scaler = load_and_preprocess_data(DATA_PATH)

In [7]:
def create_sequences_with_split(df, user_ids, seq_length, test_size=0.2):
    X_train, X_test, y_train, y_test, users_train, users_test = [], [], [], [], [], []

    for user, user_df in df.groupby('user'):
        # Разделение траектории пользователя на train/test
        split_idx = int(len(user_df) * (1 - test_size))

        # Тренировочные последовательности
        train_data = user_df.iloc[:split_idx]
        for i in range(len(train_data) - seq_length):
            X_train.append(train_data[features].values[i:i+seq_length])
            y_train.append(train_data[targets].values[i+seq_length])
            users_train.append(user_ids[user])

        # Тестовые последовательности
        test_data = user_df.iloc[split_idx-seq_length:]
        for i in range(len(test_data) - seq_length):
            X_test.append(test_data[features].values[i:i+seq_length])
            y_test.append(test_data[targets].values[i+seq_length])
            users_test.append(user_ids[user])

    return (
        np.array(X_train), np.array(X_test),
        np.array(y_train), np.array(y_test),
        np.array(users_train), np.array(users_test)
    )

In [ ]:
# Создание последовательностей с временным разделением
features = ['lat', 'lon', 'alt', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos']
targets = ['lat', 'lon']
X_train, X_test, y_train, y_test, users_train, users_test = create_sequences_with_split(df, user_ids, SEQ_LENGTH)

In [ ]:
class ContrastiveModel(tf.keras.Model):
    def __init__(self, num_users, embedding_dim):
        super().__init__()
        self.embedding = Embedding(num_users, embedding_dim)
        self.dense = Dense(embedding_dim, activation='tanh')

    def call(self, inputs):
        user_id = inputs
        user_emb = self.embedding(user_id)
        return self.dense(user_emb)

    def train_step(self, data):
        users, _ = data

        with tf.GradientTape() as tape:
            embeddings = self(users)
            anchor = embeddings[:, 0]
            positive = embeddings[:, 1]
            distances = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
            loss = self.compiled_loss(None, distances)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {'loss': loss}

In [ ]:
contrastive_model = ContrastiveModel(num_users=len(user_ids), embedding_dim=EMBEDDING_DIM)
contrastive_model.compile(optimizer=Adam(0.001), loss=tf.keras.losses.MeanSquaredError())
contrastive_model.fit(users_train, np.zeros(len(users_train)), epochs=5, batch_size=BATCH_SIZE)

In [ ]:
# Создание эмбеддингов пользователей
user_embeddings = contrastive_model.predict(np.unique(users_train))

# Объединение признаков
def combine_features(X, users, embeddings):
    embeddings_expanded = np.repeat(embeddings[users][:, np.newaxis, :], SEQ_LENGTH, axis=1)
    return np.concatenate([X, embeddings_expanded], axis=-1)

X_train_combined = combine_features(X_train, users_train, user_embeddings)
X_test_combined = combine_features(X_test, users_test, user_embeddings)

In [ ]:
def build_lstm_model(seq_length, embedding_dim, num_features, num_users):
    seq_input = Input(shape=(seq_length, num_features))
    user_input = Input(shape=(1,), dtype=tf.int32)

    user_emb = Embedding(num_users, embedding_dim)(user_input)
    user_emb = Reshape((embedding_dim,))(user_emb)

    lstm_out = LSTM(LSTM_UNITS, return_sequences=False)(seq_input)
    combined = Concatenate()([lstm_out, user_emb])

    dense_out = Dense(64, activation='relu')(combined)
    dense_out = Dense(32, activation='relu')(dense_out)
    final_output = Dense(2, activation='linear')(dense_out)

    model = Model(inputs=[seq_input, user_input], outputs=final_output)
    model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    return model

In [ ]:
model = build_lstm_model()
checkpoint = ModelCheckpoint(os.path.join(SAVE_PATH, 'best_model.h5'),
                            save_best_only=True, monitor='val_loss')
history = model.fit(
    [X_train_combined, users_train], y_train,
    validation_split=0.2,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint]
)

In [ ]:
model.load_weights(os.path.join(SAVE_PATH, 'best_model.h5'))
y_pred = model.predict([X_test_combined, users_test])

ade, fde, acc1 = calculate_metrics(y_pred, y_test)
print(f"ADE: {ade:.4f}, FDE: {fde:.4f}, Accuracy@1: {acc1:.4f}")

# Визуализация
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()